In [5]:
import openai

# OpenAI API 키 설정
client = openai.OpenAI(api_key="s")

# System prompt: 모델이 RAG 방식으로 답변할 때 반드시 인용 형식을 사용하도록 지시
system_prompt = """
당신은 GPT-4를 기반으로 하는 고급 어시스턴트입니다.
주어진 검색 결과와 질문을 바탕으로 RAG(검색 결과 기반) 답변을 생성하세요.
답변에서는 정보가 언급되는 구간마다 반드시 <co: 문서 번호>정보가 언급되는 명사구</co: 문서 번호> 형식으로 해당 내용이 어느 문서에서 인용되었는지 표기해야 합니다.
"""

In [3]:
# User prompt: 검색 결과와 질문이 포함된 전체 프롬프트
user_prompt = """
프롬프트 엔지니어링 문제

- GPT-4를 이용해서 검색 결과와 질문으로부터 RAG 답변을 얻어보세요.
- 답변에서는 정보가 언급되는 구간마다 <co: 문서 번호>정보가 언급되는 명사구</co: 문서 번호> 식으로 해당 내용이 몇 번 문서에서 인용되었는지를 표기해서 최종 답변을 작성해야 합니다.

------------------------------
검색 결과:
["Source_id": 0,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단4<노키즈존 정보제공 사이트 1><노키즈존 정보제공 사이트 2>자료 :Leavethembehind.com3.자료 :Travelandleisure.com.○어린이와 일반 고객의 탑승 공간을 구분하는 항공사도 확산 추세-2012년 말레이시아 항공은 12세 이하의 아이와 동승자는 항공기 아래층 지정구역에만 착석할 수 있도록 하는 정책 도입-에어아시아 엑스는 항공기 내에 ‘콰이엇 존(quietzone)’을 설치하고 있으며,스쿠트 항공은 2013년부터 아이들을 위한 전용좌석제 실시○영국예약사이트 ‘레이트딜’에서 비행이용승객 1,108명을 대상으로 설문조사 실시-응답자의 70%가 비행기내 노키즈존 도입을 찬성하였으며,그 중 35%는 노키즈존에 앉기 위해 추가요금도 지불할 수 있다고 응답<에어아시아의 ‘콰이엇 존’><비행 중 싫어하는 행동:앞좌석 발차기>자료 :AirAsia사이트(http://www.airasia.com/kr/ko/home.page).자료 :googleimage(https://www.google.co.kr)."]

["Source_id": 1,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단3 노키즈존은 다른 나라에서도 확산 추세영국에서는 어린이의 펍(Pub)출입을 둘러싼 논쟁이 지속되는 상황○영국은 1995년에 부모가 아이를 동반할 경우 아이도 펍에 출입할 수 있도록 법률 개정-법률 개정 전에는 만 14세 미만은 펍 출입이 허용되지 않았음○법률 개정 후 ‘부모를 동반한 어린이의 펍 출입’이 논란이 되면서 BBC는 2010년 시청자 토론방 개설-“소란스럽고 우는 아이들 때문에 분위기를 즐길 수 없다”또는 “펍은 어른들의 전유물로 남겨둬야 한다”는 의견이 압도적미국에서는 노키즈존이 민권법(CivilRightsAct)과 충돌하는지가 논란○미국에서도 어린아이를 동반한 고객들이 증가하면서 노키즈존을 도입하는 레스토랑 증가 추세○하지만 어린아이의 출입을 제한하는 것은 차별을 엄격히 금지하는 민권법에 위배된다는 주장 제기-미국 민권법은 인종이나 종교 등에 따른 차별을 엄격하게 금지하고 있는데,이에 근거해 볼 때 노키즈존도 위법하다는 주장노키즈존 정보를 알려주는 여행 사이트나 항공사도 등장○여행 사이트나 블로그를 통해 어린이 관련 정보를 제공하는 업체 증가-영국의 여행 사이트 ‘Leavethembehind.com’은 아이들의 방해를 받지 않고 휴가를 즐길 수 있는 여행 정보 제공-여행정보제공 블로그 ‘TravelandLeisures’는 아이들의 출입을 제한하는 레스토랑 리스트 제공"]

["Source_id": 2,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단7Ⅱ.노키즈존 관련 쟁점 노키즈존은 업주의 영업상의 자유에 해당하는가?어린이 고객의 출입을 제한하는 영업방침은 정부의 규제 대상이 아니라는 견해○대한민국 헌법 제 15조는 직업행사의 자유를 보장하고 있으며,이에 근거할 때 업주의 영업방침은 업주의 고유한 기본권에 해당-게다가 영업공간에서 사고가 발생했을 때 손해배상의 책임은 업주에게 귀속되기 때문에 영업방침을 규제하는 것은 과도한 재산권 침해에 해당○택시와 같이 공익성이 인정되는 서비스의 경우에는 정부의 개입이 인정되지만 카페나 음식점은 공익성이 인정되지 않는 사례에 해당-가령,택시 서비스는 공익성이 인정되고 소비자의 선택도 제한되기 때문에 법률에 의해 여객의 승차를 거부하는 행위를 금지-반면,카페나 음식점은 공익성이 인정되지도 않을 뿐만 아니라 소비자의 입장에서도 선택이 가능하기 때문에 정부의 개입은 과도한 규제에 해당노키즈존은 일반불공정 거래의 거래거절과 차별적 취급에도 해당되지 않는 사례○일부 전문가들은 노키즈존이 일반불공정 거래의 거래거절과 차별적 취급에 해당한다고 주장-공정거래법 제23조 1항 1호는 ‘부당하게 거래를 거절하거나 거래의 상대방을 차별하여 취급하는 행위’를 금지하는데,노키즈존은 정당한 이유 없는 거래거절과 차별적 취급에 해당"]

["Source_id": 3,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단1Ⅰ.노키즈존 확산 특정 장소에 아이의 출입을 금지하는 노키즈존 확산 추세아이를 동반한 고객을 받지 않겠다고 표방하는 카페나 음식점 증가○노키즈존(NoKidsZone)이란 ‘아이를 동반하고 입장할 수 없는 공간’을 의미하는 것으로 아이들의 특정 행동이나 소음을 막기 위한 조치로 확산-강남이나 홍대 등 상업지구의 카페나 음식점에서 시작해서 다른 지역으로 확산-경기도의 경우 최근 수원시나 성남시,고양시 등 어린이들이 많이 거주하는 지역을 중심으로 확산<노키즈존 시행 매장 1><노키즈존 시행 매장 2>자료 :googleimage(https://www.google.co.kr).○아이의 소란스런 행동과 부모의 방관이 노키즈존 확산의 주된 원인-카페나 음식점 등 공공장소에서 소리 지르거나 뛰어노는 아이들,이를 방치하는 부모들이 노키즈존 확산의 주된 원인으로 지목-최근에는 카페나 음식점 내에서 기저귀를 갈고 그대로 두고 간다던지,컵으로 아이의 소변을 받는 등 일부 부모의 경우 없는 행동이 논란 야기"]

["Source_id": 4,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단6노키즈존을 알고 있거나 들어본 적이 있는 응답자도 대다수○노키즈존에 대해 들어본 적이 있다고 응답한 비율은 71%로 아이들로 인해 불편을 경험한 적이 있다는 응답보다는 적지만 상당히 높은 편-성별로는 여성의 78.5%,남성의 48.4%가 노키즈존을 알고 있다고 응답했으며,자녀 유무별로는 만 10세 미만 자녀가 있는 경우 82.2%,만 10세 미만 자녀가 없는 경우 59.8%가 노키즈존을 알고 있다고 응답<전체><성별><10세 미만 자녀유무>자료 :경기연구원 모바일 설문조사(2016).○몇몇 설문조사 결과 자녀를 둔 엄마를 포함 대부분의 시민들은 노키즈존에 대해 찬성 의견 표명-2014년 9월,엄마들이 회원인 육아카페 ‘맘스홀릭베이비’에서 회원 3,525명을 대상으로 노키즈존 찬반 관련 설문을 실시한 결과 찬성이 72.7%로 압도적-2015년 9월,JTBC‘뉴스룸’에서 길거리 시민들을 대상으로 노키즈존 찬반 의견을 물은 결과도 찬성이 63%로 우세○‘알바몬’에서 아르바이트생 1,084명에게 ‘근무 중인 매장이 노키즈존으로 변경된다면 찬성할 것이냐’고 물은 결과 찬성의견이 65.5%에 달함-근무 도중 ‘유아 또는 유아를 동반한 고객으로 인해 곤란을 겪은 적이 있다’고 응답한 아르바이트생이 67.7%로,유아 고객으로 인한 업무부담 증가가 노키즈존 찬성의 주된 이유"]

질문: 노키즈존은 무엇이고, 최초로 노키즈존을 도입한 항공사는 어디인가요?
"""

In [7]:
# GPT-4o-mini 모델 호출: system과 user 메시지를 함께 전달하여 답변 생성
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0  
)

In [8]:
# 생성된 답변 출력
print(response.choices[0].message.content)

노키즈존(No Kids Zone)은 '아이를 동반하고 입장할 수 없는 공간'을 의미하며, 주로 카페나 음식점에서 아이들의 특정 행동이나 소음을 막기 위한 조치로 확산되고 있습니다<co: 문서 번호 3>노키즈존의 정의 및 확산<co: 문서 번호 3>. 

최초로 노키즈존을 도입한 항공사는 2012년 말레이시아 항공으로, 이 항공사는 12세 이하의 아이와 동승자는 항공기 아래층 지정구역에만 착석할 수 있도록 하는 정책을 도입했습니다<co: 문서 번호 0>말레이시아 항공의 노키즈존 정책<co: 문서 번호 0>. 이후 에어아시아 엑스와 스쿠트 항공도 각각 '콰이엇 존'과 아이들을 위한 전용좌석제를 도입하며 노키즈존 개념을 항공사에 적용했습니다<co: 문서 번호 0>에어아시아 엑스와 스쿠트 항공의 노키즈존 도입<co: 문서 번호 0>.


### 강사님 코드

In [9]:
system_prompt = """

당신은 검색된 문서부터 질문의 답변을 작성하는 언어 모델입니다.

### 지시사항
1. 질문으로부터 주어진 다수의 문서인 검색 결과에서 답을 찾아 작성하세요.
2. 검색된 문서에 질문의 답이 없는 경우에는 질문에 대한 내용을 언급하면서 답을 찾을 수 없다고 작성하세요.
3. 검색 결과의 문서에서 답변을 인용한 경우 인용한 부분을 <co: Source_id> 와 </co: Source_id> 기호를 사용해서 감싸서 표시하세요. 감싸지 않고서는 정보를 전달하려 하지 마십시오.
4. 예를 들어 답변을 Source_id : 0 에서 인용했다면 <co: 0>답변</co: 0> 처럼 표기합니다.
5. 그 어떠한 정보도 인용없이 설명하려 하지마십시오. <co: Source_id> </co: Source_id> 태그 외부에서는 그 어떠한 정보 전달도 허용하지 않겠습니다.
6. <co: Source_id> </co: Source_id> 태그 외부에서는 수식어 외에는 사용해서는 안 됩니다. 정보 작성이 불가능합니다.
7. <co: Source_id> </co: Source_id> 태그 내부에서만 정보 전달이 가능합니다. 다시 말해 몇 번 문서에서 인용했는지 출처를 남기지 않으면 그 어떠한 정보 전달을 허용하지 않습니다.
8. 경고합니다. <co: Source_id> </co: Source_id> 태그 외부에서는 정보 전달이 불가능합니다. 사용자의 질문이나 간단한 형용사 외에는 태그 외적으로 언급하지 마십시오.
9. 답변은 질문과 관계있다면 최대한 많은 문서를 인용하여 최대한 풍부하고 장문의 답변을 작성하십시오.

예시)
검색 결과:
["Source_id": 0, "Content": "황제 펭귄은 세계에서 가장 키가 큰 펭귄입니다. 또한, 삶의 대부분을 남극해에서 보내며, 모든 새 중 가장 깊은 수심인 500~600m 까지 잠수할 수 있다. 주로 작은 어류와 오징어를 사냥합니다."]
["Source_id": 1, "Content": "황제 펭귄은 남극에서 주로 서식하며 성체의 키가 120cm, 수명은 약 20년, 체중이 23kg에서 최대 45kg에까지 달합니다."]
["Source_id": 2, "Content": "꼬마펭귄 핑구에 등장하는 핑구의 여동생 핑가가 황제펭귄 새끼와 비슷하게 생겼다. 그 외에도 단역으로 나오는 아기 펭귄들도 황제펭귄 새끼의 모습이다. 그런데 핑구를 비롯한 나머지 펭귄들은 황제펭귄 특유의 귀의 노란 무늬가 없는 것으로 보아 아델리 펭귄을 모티브로 한 것으로 보인다. 참고로 아델리 펭귄과 황제펭귄은 서로 숙적이자 견원지간 사이므로 서로 만나려고 하지 않는다. 그나마 아빠는 황제펭귄과 외형이 닮았다. 그런데 잘 보면 핑구도 그렇고 가족 전부 흰 배가 노란끼가 살짝 도는 걸 보면 황제펭귄이 맞는 걸로 보인다."]
질문: 남극에서 가장 큰 펭귄은?
답변: 남극에서 가장 큰 펭귄은 <co: 0>황제 펭귄이며 세계에서 가장 키가 큰 펭귄</co: 0>입니다. <co: 1>남극에서 주로 서식</co: 1>하며 <co: 1>성체의 키가 120cm 체중이 23kg에서 최대 45kg</co: 1>에까지 달합니다.
"""

user_prompt = """
검색 결과:
["Source_id": 0,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단4<노키즈존 정보제공 사이트 1><노키즈존 정보제공 사이트 2>자료 :Leavethembehind.com3.자료 :Travelandleisure.com.○어린이와 일반 고객의 탑승 공간을 구분하는 항공사도 확산 추세-2012년 말레이시아 항공은 12세 이하의 아이와 동승자는 항공기 아래층 지정구역에만 착석할 수 있도록 하는 정책 도입-에어아시아 엑스는 항공기 내에 ‘콰이엇 존(quietzone)’을 설치하고 있으며,스쿠트 항공은 2013년부터 아이들을 위한 전용좌석제 실시○영국예약사이트 ‘레이트딜’에서 비행이용승객 1,108명을 대상으로 설문조사 실시-응답자의 70%가 비행기내 노키즈존 도입을 찬성하였으며,그 중 35%는 노키즈존에 앉기 위해 추가요금도 지불할 수 있다고 응답<에어아시아의 ‘콰이엇 존’><비행 중 싫어하는 행동:앞좌석 발차기>자료 :AirAsia사이트(http://www.airasia.com/kr/ko/home.page).자료 :googleimage(https://www.google.co.kr)."]
["Source_id": 1,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단3 노키즈존은 다른 나라에서도 확산 추세영국에서는 어린이의 펍(Pub)출입을 둘러싼 논쟁이 지속되는 상황○영국은 1995년에 부모가 아이를 동반할 경우 아이도 펍에 출입할 수 있도록 법률 개정-법률 개정 전에는 만 14세 미만은 펍 출입이 허용되지 않았음○법률 개정 후 ‘부모를 동반한 어린이의 펍 출입’이 논란이 되면서 BBC는 2010년 시청자 토론방 개설-“소란스럽고 우는 아이들 때문에 분위기를 즐길 수 없다”또는 “펍은 어른들의 전유물로 남겨둬야 한다”는 의견이 압도적미국에서는 노키즈존이 민권법(CivilRightsAct)과 충돌하는지가 논란○미국에서도 어린아이를 동반한 고객들이 증가하면서 노키즈존을 도입하는 레스토랑 증가 추세○하지만 어린아이의 출입을 제한하는 것은 차별을 엄격히 금지하는 민권법에 위배된다는 주장 제기-미국 민권법은 인종이나 종교 등에 따른 차별을 엄격하게 금지하고 있는데,이에 근거해 볼 때 노키즈존도 위법하다는 주장노키즈존 정보를 알려주는 여행 사이트나 항공사도 등장○여행 사이트나 블로그를 통해 어린이 관련 정보를 제공하는 업체 증가-영국의 여행 사이트 ‘Leavethembehind.com’은 아이들의 방해를 받지 않고 휴가를 즐길 수 있는 여행 정보 제공-여행정보제공 블로그 ‘TravelandLeisures’는 아이들의 출입을 제한하는 레스토랑 리스트 제공"]
["Source_id": 2,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단7Ⅱ.노키즈존 관련 쟁점 노키즈존은 업주의 영업상의 자유에 해당하는가?어린이 고객의 출입을 제한하는 영업방침은 정부의 규제 대상이 아니라는 견해○대한민국 헌법 제 15조는 직업행사의 자유를 보장하고 있으며,이에 근거할 때 업주의 영업방침은 업주의 고유한 기본권에 해당-게다가 영업공간에서 사고가 발생했을 때 손해배상의 책임은 업주에게 귀속되기 때문에 영업방침을 규제하는 것은 과도한 재산권 침해에 해당○택시와 같이 공익성이 인정되는 서비스의 경우에는 정부의 개입이 인정되지만 카페나 음식점은 공익성이 인정되지 않는 사례에 해당-가령,택시 서비스는 공익성이 인정되고 소비자의 선택도 제한되기 때문에 법률에 의해 여객의 승차를 거부하는 행위를 금지-반면,카페나 음식점은 공익성이 인정되지도 않을 뿐만 아니라 소비자의 입장에서도 선택이 가능하기 때문에 정부의 개입은 과도한 규제에 해당노키즈존은 일반불공정 거래의 거래거절과 차별적 취급에도 해당되지 않는 사례○일부 전문가들은 노키즈존이 일반불공정 거래의 거래거절과 차별적 취급에 해당한다고 주장-공정거래법 제23조 1항 1호는 ‘부당하게 거래를 거절하거나 거래의 상대방을 차별하여 취급하는 행위’를 금지하는데,노키즈존은 정당한 이유 없는 거래거절과 차별적 취급에 해당"]
["Source_id": 3,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단1Ⅰ.노키즈존 확산 특정 장소에 아이의 출입을 금지하는 노키즈존 확산 추세아이를 동반한 고객을 받지 않겠다고 표방하는 카페나 음식점 증가○노키즈존(NoKidsZone)이란 ‘아이를 동반하고 입장할 수 없는 공간’을 의미하는 것으로 아이들의 특정 행동이나 소음을 막기 위한 조치로 확산-강남이나 홍대 등 상업지구의 카페나 음식점에서 시작해서 다른 지역으로 확산-경기도의 경우 최근 수원시나 성남시,고양시 등 어린이들이 많이 거주하는 지역을 중심으로 확산<노키즈존 시행 매장 1><노키즈존 시행 매장 2>자료 :googleimage(https://www.google.co.kr).○아이의 소란스런 행동과 부모의 방관이 노키즈존 확산의 주된 원인-카페나 음식점 등 공공장소에서 소리 지르거나 뛰어노는 아이들,이를 방치하는 부모들이 노키즈존 확산의 주된 원인으로 지목-최근에는 카페나 음식점 내에서 기저귀를 갈고 그대로 두고 간다던지,컵으로 아이의 소변을 받는 등 일부 부모의 경우 없는 행동이 논란 야기"]
["Source_id": 4,"Content": "노키즈존 확산, 어떻게 볼 것인가?이슈 & 진단6노키즈존을 알고 있거나 들어본 적이 있는 응답자도 대다수○노키즈존에 대해 들어본 적이 있다고 응답한 비율은 71%로 아이들로 인해 불편을 경험한 적이 있다는 응답보다는 적지만 상당히 높은 편-성별로는 여성의 78.5%,남성의 48.4%가 노키즈존을 알고 있다고 응답했으며,자녀 유무별로는 만 10세 미만 자녀가 있는 경우 82.2%,만 10세 미만 자녀가 없는 경우 59.8%가 노키즈존을 알고 있다고 응답<전체><성별><10세 미만 자녀유무>자료 :경기연구원 모바일 설문조사(2016).○몇몇 설문조사 결과 자녀를 둔 엄마를 포함 대부분의 시민들은 노키즈존에 대해 찬성 의견 표명-2014년 9월,엄마들이 회원인 육아카페 ‘맘스홀릭베이비’에서 회원 3,525명을 대상으로 노키즈존 찬반 관련 설문을 실시한 결과 찬성이 72.7%로 압도적-2015년 9월,JTBC‘뉴스룸’에서 길거리 시민들을 대상으로 노키즈존 찬반 의견을 물은 결과도 찬성이 63%로 우세○‘알바몬’에서 아르바이트생 1,084명에게 ‘근무 중인 매장이 노키즈존으로 변경된다면 찬성할 것이냐’고 물은 결과 찬성의견이 65.5%에 달함-근무 도중 ‘유아 또는 유아를 동반한 고객으로 인해 곤란을 겪은 적이 있다’고 응답한 아르바이트생이 67.7%로,유아 고객으로 인한 업무부담 증가가 노키즈존 찬성의 주된 이유"]
질문: 노키즈존은 무엇이고, 최초로 노키즈존을 도입한 항공사는 어디인가요?"""

response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ],
  temperature=0
)

print(response.choices[0].message.content)

노키즈존은 <co: 3>'아이를 동반하고 입장할 수 없는 공간'</co: 3>을 의미하며, <co: 3>아이들의 특정 행동이나 소음을 막기 위한 조치로 확산</co: 3>되고 있습니다. 최초로 노키즈존을 도입한 항공사는 <co: 0>2012년 말레이시아 항공</co: 0>으로, <co: 0>12세 이하의 아이와 동승자는 항공기 아래층 지정구역에만 착석할 수 있도록 하는 정책을 도입</co: 0>했습니다.
